# 라이브러리 설치

In [4]:
!pip install openai langchain langchain-community langchain-core langchain_openai langchain-experimental --q


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# 동적 프롬프트

In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

import os
from pathlib import Path
from dotenv import load_dotenv

def load_env_from_project_root():
    """
    현재 실행 중인 스크립트(LLM_basic.ipynb)에서
    step10_LLM 디렉토리의 .env 파일을 로드
    """
    # 현재 파일 위치
    current_path = Path.cwd()
    
    # step10_LLM 디렉토리로 이동 (../../)
    project_root = current_path.parent.parent
    
    # .env 파일 경로
    env_path = project_root / '.env'
    
    # .env 파일이 존재하는지 확인
    if env_path.exists():
        load_dotenv(env_path)
        print(f"Loaded .env from: {env_path}")
        return True
    else:
        raise FileNotFoundError(f".env file not found at: {env_path}")

# 실행
try:
    load_env_from_project_root()
    
    # 환경 변수 사용
    api_key = os.getenv('OPENAI_API_KEY')
    if api_key:
        print("API Key loaded successfully")
        # API 키 마스킹하여 출력
        masked_key = f"{api_key[:8]}...{api_key[-4:]}"
        print(f"API Key: {masked_key}")
    else:
        print("API Key not found in .env file")
except Exception as e:
    print(f"Error loading .env file: {e}")

Loaded .env from: c:\Users\campus3S043\Desktop\alpaco_lectures\step10_LLM\.env
API Key loaded successfully
API Key: sk-proj-...yTUA


# 아래 코드 설명
- 프롬프트 템플릿을 만들어서 질문을 입력받고 답변을 생성하는 체인을 구성합니다.
- 체인은 다음과 같이 구성됩니다:
  1. PromptTemplate: 질문을 받아 포맷팅
  2. ChatOpenAI: GPT 모델을 통한 답변 생성 
  3. StrOutputParser: 출력을 문자열로 파싱

In [7]:
template = """Question: {question} 단계별로 생각해봅시다.
Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])
chain = prompt | ChatOpenAI(model_name="gpt-4o-mini") | StrOutputParser()
question = "2016년 올림픽이 열린 국가의 수도는 인구가 얼마인가요?"
print(chain.invoke(question))

2016년 올림픽은 브라질의 리우데자네이루에서 열렸습니다. 이제 단계별로 리우데자네이루의 인구에 대해 알아보겠습니다.

1. **올림픽 개최 도시 확인**: 2016년 여름 올림픽은 리우데자네이루에서 진행되었습니다.
  
2. **리우데자네이루의 수도 역할 확인**: 리우데자네이루는 브라질의 수도가 아닙니다. 브라질의 현재 수도는 브라질리아(Brasília)입니다. 하지만 리우데자네이루는 과거에 브라질의 수도였던 도시입니다.

3. **인구 조사**: 리우데자네이루의 인구는 2020년 기준으로 약 670만 명 정도입니다. 최근 인구 통계나 다른 지표에 따라 수치는 조금 달라질 수 있습니다.

결론적으로, 2016년 올림픽이 열린 리우데자네이루는 현재 약 670만 명의 인구를 가지고 있습니다.


# 에이전트와 도구

In [11]:
!pip install wikipedia numexpr --q


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_react_agent, AgentExecutor

from langchain_community.agent_toolkits.load_tools import load_tools
from langchain import hub

load_env_from_project_root()

llm = ChatOpenAI(model_name="gpt-4o-mini")
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = create_react_agent(
    tools=tools,
    llm=llm,
    prompt=hub.pull("hwchase17/react"),
)

# 복잡한 문제에 대한 추론 능력을 보여주고 있음
question = "2023 럭비 월드컵에서 우승한 나라의 인구의 제곱근은 얼마인가요?"
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": question})

Loaded .env from: c:\Users\campus3S043\Desktop\alpaco_lectures\step10_LLM\.env


c:\Users\campus3S043\Desktop\alpaco_lectures\venv\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
The first step is to find out which country won the 2023 Rugby World Cup. Once I have that information, I will then need to find the population of that country to calculate the square root of its population. 

Action: wikipedia  
Action Input: 2023 Rugby World Cup winner  
Page: 2023 Rugby World Cup
Summary: The 2023 Rugby World Cup (French: Coupe du monde de rugby 2023) was the  tenth men's Rugby World Cup, the quadrennial world championship for national rugby union teams. It took place in France from 8 September to 28 October 2023 in nine venues across the country. The opening game and final took place at the Stade de France, north of Paris. The tournament was held in the bicentenary year of the purported invention of the sport by William Webb Ellis.
The tournament was scheduled to last six weeks, but in February 2021 World Rugby added a week to provide additional rest days for player welfare. This meant that teams had a minimum of five days' r

{'input': '2023 럭비 월드컵에서 우승한 나라의 인구의 제곱근은 얼마인가요?', 'output': '약 7874.01'}

# 랭스미스 : 메모리 기능 지원

In [14]:
!pip install langgraph --q



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
import uuid

load_env_from_project_root()
llm = ChatOpenAI(model_name="gpt-4o-mini")

# 상태 그래프 정의
workflow = StateGraph(state_schema=MessagesState)

# llm 호출 함수 정의
def call_llm(state: MessagesState):
    response = llm.invoke(state["messages"])
    return {"messages": response}

# 그래프에 llm 추가
workflow.add_edge(START, "llm")
workflow.add_node("llm", call_llm)

# 메모리 저장
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# 스레드 ID 생성
thread_id = uuid.uuid4()

# 스레드 ID 지정
config = {"configurable": {"thread_id": thread_id}}

# 대화 시작
query = "안녕하세요. 저는 홍길동입니다."

input_messages = [
    {
        "role": "system",
        "content": """당신은 상점을 운영하는 상인입니다. 
        고객의 질문에 성실히 답하세요.""",
    },
    {"role": "user", "content": query},
]
for event in app.stream({"messages": input_messages}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

# 새로운 대화
new_query = "제 이름을 기억하나요?"
new_input_messages = [
    {"role": "user", "content": new_query},
]

for event in app.stream({"messages": new_input_messages}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()


Loaded .env from: c:\Users\campus3S043\Desktop\alpaco_lectures\step10_LLM\.env
================================ Human Message =================================

안녕하세요. 저는 홍길동입니다.
================================== Ai Message ==================================

안녕하세요, 홍길동님! 저희 상점에 오신 것을 환영합니다. 어떤 도움을 드릴 수 있을까요? 필요하신 상품이나 정보가 있으시면 말씀해 주세요.
================================ Human Message =================================

제 이름을 기억하나요?
================================== Ai Message ==================================

네, 홍길동님이라고 말씀하셨습니다. 고객님의 이름을 기억하는 것은 중요하다고 생각합니다! 더 궁금하신 점이나 필요하신 것이 있으면 언제든지 말씀해 주세요.


# 임베딩

In [22]:
!pip install pypdf faiss-cpu --q


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
from pathlib import Path
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain.chains.retrieval_qa.base import RetrievalQA

load_env_from_project_root()

# file_path = Path(__file__).parent.joinpath("files").joinpath("ExplorersGuide.pdf")
file_path = "files/ExplorersGuide.pdf"
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()

embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(pages, embeddings)

q = "링크의 전형적인 의상 색깔은 무엇인가요?"
print(db.similarity_search(q)[0])

llm = ChatOpenAI(model_name="gpt-4o-mini")
chain = RetrievalQA.from_llm(llm=llm, retriever=db.as_retriever())

print(chain.invoke(q, return_only_outputs=True)['result'])


Loaded .env from: c:\Users\campus3S043\Desktop\alpaco_lectures\step10_LLM\.env
page_content='While Link’s traditional green 
tunic is certainly an iconic look, his 
wardrobe has expanded quite a bit 
in his latest adventure. How you 
dress isn’t just an expression of 
personal style either. In addition 
to upping your defense a bit, 
different outﬁts have different 
effects that will prove useful 
depending on what you’re doing 
and where you’re going. The Warm 
Doublet, for example, is a fantastic 
fusion of fashion and function. It 
not only looks rather spiffy, it also 
protects you from the cold, allow-
ing you to explore snowy areas 
without losing health along the 
way. The Climber’s set ups your 
climbing speed, which means 
you’ll be able to climb farther than 
you normally would 
before running out of 
stamina. The Stealth set 
increases your stealth, 
so it’s handy when 
you’re out hunting or 
trying to sneak up on 
enemies. The Zora 
Armor not only helps 
you swim faster, it

# 라마인덱스

In [4]:
!pip install llama-index-core llama-index-readers-file llama-index-llms-ollama llama-index-embeddings-huggingface


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from dotenv import load_dotenv
from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

load_env_from_project_root()

# 데이터 폴더의 문서를 로드 및 색인
llm = OpenAI(model_name="gpt-4o-mini")
documents = SimpleDirectoryReader("files").load_data()
index = VectorStoreIndex.from_documents(documents, llm=llm)

# 쿼리 작성
query_engine = index.as_query_engine()
response = query_engine.query("링크의 전형적인 의상 색깔은 무엇인가요?")
print(response)

Loaded .env from: c:\Users\campus3S043\Desktop\alpaco_lectures\step10_LLM\.env
Link의 전형적인 의상 색깔은 녹색입니다.


# 라마인덱스 벡터

In [20]:
!pip install llama-index-vector-stores-faiss



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
from dotenv import load_dotenv
from llama_index.core import (SimpleDirectoryReader, StorageContext,
                              VectorStoreIndex)
from llama_index.core.settings import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.vector_stores.faiss import FaissVectorStore
import faiss

load_env_from_project_root()

Settings.llm = OpenAI(model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding()

# FAISS 인덱스 생성 (1536은 OpenAI 임베딩 차원)
dimension = 1536
faiss_index = faiss.IndexFlatL2(dimension)

# FAISS 벡터 저장소 생성
vector_store = FaissVectorStore(faiss_index=faiss_index)

# 임베딩을 위한 스토리지 세팅
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# 데이터 폴더 내의 문서를 로드 및 색인
documents = SimpleDirectoryReader("files").load_data()
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context)

# 쿼리 작성
query_engine = index.as_query_engine()
response = query_engine.query("링크의 전형적인 의상 색깔은 무엇인가요?")
print(response)

Loaded .env from: c:\Users\campus3S043\Desktop\alpaco_lectures\step10_LLM\.env


c:\Users\campus3S043\Desktop\alpaco_lectures\venv\Lib\site-packages\llama_index\core\indices\base.py:110: DeprecationWarning: Call to deprecated method get_doc_id. ('get_doc_id' is deprecated, access the 'id_' property instead.) -- Deprecated since version 0.12.2.
  docstore.set_document_hash(doc.get_doc_id(), doc.hash)


링크의 전형적인 의상 색깔은 녹색입니다.


# 어시스턴트 API

In [25]:
import time
from openai import OpenAI
from dotenv import load_dotenv

load_env_from_project_root()
client = OpenAI()

def waiting_assistant_in_progress(thread_id, run_id, max_loops=20):
    for _ in range(max_loops):
        run = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run_id
        )
        if run.status != 'in_progress':
            break
        time.sleep(1)
    return run

# 파일 추가 완료 함수
def waiting_file_batch_in_progress(file_batch, max_loops=20):
    for _ in range(max_loops):
        if file_batch.status != 'in_progress':
            break
        time.sleep(1)
    return file_batch

# 벡터 저장소 "Explorers Guide" 생성
vector_store = client.beta.vector_stores.create(name="Explorers Guide")
 
# PDF 파일 업로드
file_streams = [open('files/ExplorersGuide.pdf', "rb")]
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

# 파일 업로드 완료 대기
file_batch = waiting_file_batch_in_progress(file_batch)

# 어시스턴트 생성
zelda_expert_assistant = client.beta.assistants.create(
    name='Zelda expert',
    instructions='''당신은 비디오 게임 젤다의 전문가이며, 제가 드린 파일을 사용하여 게임에 대한 질문에 답해 주셔야 합니다.''',
    model='gpt-4o',
    tools=[{'type': 'file_search'}],
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
)


# 스레드 생성
thread = client.beta.threads.create()

# 스레드에 메시지 추가
client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content='링크의 전형적인 의상 색깔은 무엇인가요?'
)

# 스레드 실행
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=zelda_expert_assistant.id
)

# 스레드 실행 완료 후 결과 출력
run = waiting_assistant_in_progress(thread.id, run.id)
messages = client.beta.threads.messages.list(thread_id=thread.id)
print(messages.data[0].content[0].text.value)


Loaded .env from: c:\Users\campus3S043\Desktop\alpaco_lectures\step10_LLM\.env
링크의 전형적인 의상 색깔은 무엇인가요?


In [26]:
import time, json
from openai import OpenAI
from dotenv import load_dotenv

load_env_from_project_root()
client = OpenAI()

def waiting_assistant_in_progress(thread_id, run_id, max_loops=20):
    for _ in range(max_loops):
        run = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run_id
        )
        if (run.status == 'requires_action'
            and run.required_action.type == 'submit_tool_outputs'):

            tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
            if tool_call.function.name == 'getCurrentTemperature':
                arguments = json.loads(tool_call.function.arguments)
                fct_output = getCurrentTemperature(arguments['city'])
            else:
                raise Exception('Unexpected function')

            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=[
                    {
                        'tool_call_id': tool_call.id,
                        'output': fct_output
                    }
                ]
            )
        time.sleep(1)
    return run



def getCurrentTemperature(city):
    return str(len(city)) + '°C'

function = {
    'name': 'getCurrentTemperature',
    'description': '도시의 현재 기온을 가져오는 함수',
    'parameters': {
        'type': 'object',
        'properties': {
            'city': {
                'type': 'string',
                'description':
                '기온을 가져올 도시의 이름.'
                '예: ‘뉴욕’, ‘런던’, 등'
            }
        },
        'required': ['city']
    }
}

tools = [{
    'type': 'function',
            'function': function
}]

client = OpenAI()
assistant = client.beta.assistants.create(
    name='Weather Assistant',
    instructions='당신은 외부 도구를 사용하여 도시의 현재 기온을 제공하는 어시스턴트입니다.',
    model='gpt-4o-mini',
    tools=tools
)

thread = client.beta.threads.create()

client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content='베를린의 현재 기온은 얼마인가요?'
)

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

run = waiting_assistant_in_progress(thread.id, run.id)
messages = client.beta.threads.messages.list(thread_id=thread.id)
print(messages.data[0].content[0].text.value)


Loaded .env from: c:\Users\campus3S043\Desktop\alpaco_lectures\step10_LLM\.env
베를린의 현재 기온은 3°C입니다.
